In [1]:
from Bio import SeqIO
import numpy as np
import csv
from shared import *
from project import Aligner
from time import time

In [2]:
max_id_len = 60
reads = np.zeros((1575, 2), dtype='S{:d}'.format(max_id_len))
i = 0
for seq_record in SeqIO.parse("reads.fa", "fasta"):
    reads[i, 0] = seq_record.id
    reads[i, 1] = str(seq_record.seq)
    i += 1

In [3]:
genome = None
for seq_record in SeqIO.parse("genome.fa", "fasta"):
    genome = str(seq_record.seq)

In [4]:
len(genome)

10949000

In [5]:
genes, isoforms, exons = {}, {}, {}
with open("genes.tab") as tsv:
    for line in csv.reader(tsv, dialect="excel-tab"):
        name = line[0].split('_')[-1]
        if name == 'gene':
            genes[line[1]] = line[2].split(';')
        elif name == 'isoform':
            isoforms[line[1]] = line[2].split(';')
        elif name == 'exon':
            exons[line[1]] = Exon(line[1], int(line[2]), int(line[3]))
# Create the Isoform objects
for k in isoforms:
    isoforms[k] = Isoform(k, [exons[key] for key in isoforms[k]])

# Create the Genes objects
for k in genes:
    genes[k] = Gene(k, [isoforms[key] for key in genes[k]])


In [ ]:
t = time()
a = Aligner(genome, genes)
t = time() - t
print("time to run Aligner.__init__: " + str(t))

updated
